In [27]:
import pandas as pd

In [32]:
df = pd.read_csv('raw_books.csv')
df.head()

,CLIENTE,PRODUCTO,AUTOR,CALIFICACION,CATEGORIA
0,Alfonso,Un mundo feliz,Aldous Huxley,10,ficcion
1,Alfonso,La dama de las camelias,Alexandre Dumas,10,novela
2,Alfonso,El lazarillo de Tormes,Anónimo,9,novela
3,Alfonso,El Principito,Antoine de Sainte-Exupéry,10,fantasia
4,Alfonso,Nacidos para correr,Christopher McDougall,10,deportes


# books_keyspace

## s-04-insert-books-by-title.cql

In [14]:
new_df = df.loc[:, ["PRODUCTO", "AUTOR", "CATEGORIA"]]
new_df = new_df.rename(columns={'PRODUCTO':'book_title', 'AUTOR':'authors', 'CATEGORIA':'book_category'})
new_df.head()

,book_title,authors,book_category
0,Un mundo feliz,Aldous Huxley,ficcion
1,La dama de las camelias,Alexandre Dumas,novela
2,El lazarillo de Tormes,Anónimo,novela
3,El Principito,Antoine de Sainte-Exupéry,fantasia
4,Nacidos para correr,Christopher McDougall,deportes


In [15]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   book_title     193 non-null    object
 1   authors        193 non-null    object
 2   book_category  193 non-null    object
dtypes: object(3)
memory usage: 4.6+ KB


In [30]:
new_rows = []
base = "INSERT INTO books_by_title(book_title, book_category, authors) VALUES ("
for idx, row in new_df.iterrows():
    values = "\'" + row["book_title"] + "\', \'" + row["book_category"] + "\', {\'" + row["authors"] + "\'}"
    new_rows.append(base + values + ");")
final_df = pd.DataFrame(data = {'statement': new_rows})
final_df.head()

,statement
0,"INSERT INTO books_by_title(book_title, book_ca..."
1,"INSERT INTO books_by_title(book_title, book_ca..."
2,"INSERT INTO books_by_title(book_title, book_ca..."
3,"INSERT INTO books_by_title(book_title, book_ca..."
4,"INSERT INTO books_by_title(book_title, book_ca..."


In [25]:
final_df.to_csv('s-04-insert-books-by-title.cql', index=False)

## s-04-insert-books-by-username.cql

In [29]:
df.head()

,CLIENTE,PRODUCTO,AUTOR,CALIFICACION,CATEGORIA
0,Alfonso,Un mundo feliz,Aldous Huxley,10,ficcion
1,Alfonso,La dama de las camelias,Alexandre Dumas,10,novela
2,Alfonso,El lazarillo de Tormes,Anónimo,9,novela
3,Alfonso,El Principito,Antoine de Sainte-Exupéry,10,fantasia
4,Alfonso,Nacidos para correr,Christopher McDougall,10,deportes


In [39]:
new_rows = []
base = "INSERT INTO books_by_username(username, book_title, book_category, book_authors, score) VALUES ("
for idx, row in df.iterrows():
    sep = "\', \'"
    unique = "\'"
    dict_sep = "\', {\'"
    values = unique + row["CLIENTE"] + sep + row["PRODUCTO"] + sep + row["CATEGORIA"] + dict_sep + row["AUTOR"] + '\'},' + str(row["CALIFICACION"])
    new_rows.append(base + values + ");")
final_df = pd.DataFrame(data = {'statement': new_rows})
final_df.head()

,statement
0,"INSERT INTO books_by_username(username, book_t..."
1,"INSERT INTO books_by_username(username, book_t..."
2,"INSERT INTO books_by_username(username, book_t..."
3,"INSERT INTO books_by_username(username, book_t..."
4,"INSERT INTO books_by_username(username, book_t..."


In [40]:
final_df.to_csv('s-04-insert-books-by-username.cql', index=False)

## s-04-insert-books-by-score.cql

In [42]:
df.head()

,CLIENTE,PRODUCTO,AUTOR,CALIFICACION,CATEGORIA
0,Alfonso,Un mundo feliz,Aldous Huxley,10,ficcion
1,Alfonso,La dama de las camelias,Alexandre Dumas,10,novela
2,Alfonso,El lazarillo de Tormes,Anónimo,9,novela
3,Alfonso,El Principito,Antoine de Sainte-Exupéry,10,fantasia
4,Alfonso,Nacidos para correr,Christopher McDougall,10,deportes


In [45]:
new_rows = []
base = "INSERT INTO books_by_score(score, book_title, book_category, authors) VALUES ("
sep = "\', \'"
unique = "\'"
dict_sep = "\', {\'"
end_dict_sep = "\'}"
for idx, row in df.iterrows():
    values = str(row["CALIFICACION"]) + ',\'' + row["PRODUCTO"] + sep + row["CATEGORIA"] + dict_sep + row["AUTOR"] + end_dict_sep
    new_rows.append(base + values + ");")
final_df = pd.DataFrame(data = {'statement': new_rows})
final_df.head()

,statement
0,"INSERT INTO books_by_score(score, book_title, ..."
1,"INSERT INTO books_by_score(score, book_title, ..."
2,"INSERT INTO books_by_score(score, book_title, ..."
3,"INSERT INTO books_by_score(score, book_title, ..."
4,"INSERT INTO books_by_score(score, book_title, ..."


In [46]:
final_df.to_csv('s-04-insert-books-by-score.cql', index=False)

# clients_keyspace

## s-05-insert-clients-by-username.cql

In [7]:
clients_df = pd.read_csv('raw_clients.csv')
clients_df.columns = clients_df.columns.str.replace(' ', '')
for col in clients_df.columns:
    if col != "TELEFONO":
        clients_df[col] = clients_df[col].str.replace(' ', '')
    if col == "CLIENTE":
        clients_df[col] = clients_df[col].str.lower()
clients_df.head()

,CLIENTE,NOMBRE,APELLIDO,PAIS,MEMBRESIA,TELEFONO
0,alfonso,Alfonso,Barajas,Mexico,Estándar,5539848129
1,antonio,Antonio,Aguilar,México,Estándar,5539299465
2,artemio,Artemio,Padilla,Nigeria,Diamante,5539852382
3,avilix,Avilix,Hernández,México,Estándar,5539852756
4,benito,Benito,Franco,Nigeria,Estándar,5539448736


In [85]:
new_rows = []
base = "INSERT INTO clients_by_username(username, last_name, country, first_name, membership) VALUES ("
sep = "\', \'"
unique = "\'"
dict_sep = "\', {\'"
end_dict_sep = "\'}"
for idx, row in clients_df.iterrows():
    values = unique + row["CLIENTE"] + sep + row["APELLIDO"] + sep + row["PAIS"] + sep + row["NOMBRE"] + sep + row["MEMBRESIA"] + unique
    new_rows.append(base + values + ");")
final_df = pd.DataFrame(data = {'statement': new_rows})
final_df.head()

,statement
0,"INSERT INTO clients_by_username(username, last..."
1,"INSERT INTO clients_by_username(username, last..."
2,"INSERT INTO clients_by_username(username, last..."
3,"INSERT INTO clients_by_username(username, last..."
4,"INSERT INTO clients_by_username(username, last..."


In [86]:
final_df.to_csv('s-05-insert-clients-by-username.cql', index=False)

## s-05-insert-clients-by-country.cql

In [88]:
new_rows = []
base = "INSERT INTO clients_by_country(country, username, first_name, last_name, membership) VALUES ("
sep = "\', \'"
unique = "\'"
dict_sep = "\', {\'"
end_dict_sep = "\'}"
for idx, row in clients_df.iterrows():
    values = unique + row["PAIS"] + sep + row["CLIENTE"] + sep + row["NOMBRE"] + sep + row["APELLIDO"] + sep + row["MEMBRESIA"] + unique
    new_rows.append(base + values + ");")
final_df = pd.DataFrame(data = {'statement': new_rows})
print(final_df.iloc[0, :].to_numpy())
final_df.head()

["INSERT INTO clients_by_country(country, username, first_name, last_name, membership) VALUES ('Mexico', 'alfonso', 'Alfonso', 'Barajas', 'Estándar');"]


,statement
0,"INSERT INTO clients_by_country(country, userna..."
1,"INSERT INTO clients_by_country(country, userna..."
2,"INSERT INTO clients_by_country(country, userna..."
3,"INSERT INTO clients_by_country(country, userna..."
4,"INSERT INTO clients_by_country(country, userna..."


In [89]:
final_df.to_csv('s-05-insert-clients-by-country.cql', index=False)

## s-04-insert-category-by-name.cql

In [18]:
categories_df = pd.read_csv('raw_categories.csv')
categories_df.head()

,CATEGORIA,FRECUENCIA
0,ajedrez,2
1,autoayuda,2
2,biografia,2
3,comedia,2
4,deportes,3


In [19]:
new_rows = []
base = "INSERT INTO category_by_name(category) VALUES ("
sep = "\', \'"
unique = "\'"
dict_sep = "\', {\'"
end_dict_sep = "\'}"
for idx, row in categories_df.iterrows():
    values = unique + row["CATEGORIA"] + unique
    new_rows.append(base + values + ");")
final_df = pd.DataFrame(data = {'statement': new_rows})
print(final_df.iloc[0, :].to_numpy())
final_df.head()

["INSERT INTO category_by_name(category) VALUES ('ajedrez');"]


,statement
0,INSERT INTO category_by_name(category) VALUES ...
1,INSERT INTO category_by_name(category) VALUES ...
2,INSERT INTO category_by_name(category) VALUES ...
3,INSERT INTO category_by_name(category) VALUES ...
4,INSERT INTO category_by_name(category) VALUES ...


In [20]:
final_df.to_csv('s-04-insert-category-by-name.cql', index=False)

## s-05-insert-clients-by-membership.cql

In [92]:
new_rows = []
base = "INSERT INTO clients_by_membership(membership, username, first_name, last_name, country) VALUES ("
sep = "\', \'"
unique = "\'"
dict_sep = "\', {\'"
end_dict_sep = "\'}"
for idx, row in clients_df.iterrows():
    values = unique + row["MEMBRESIA"] + sep + row["CLIENTE"] + sep + row["NOMBRE"] + sep + row["APELLIDO"] + sep + row["PAIS"] + unique
    new_rows.append(base + values + ");")
final_df = pd.DataFrame(data = {'statement': new_rows})
print(final_df.iloc[0, :].to_numpy())
final_df.head()

["INSERT INTO clients_by_membership(membership, username, first_name, last_name, country) VALUES ('Estándar', 'alfonso', 'Alfonso', 'Barajas', 'Mexico');"]


,statement
0,"INSERT INTO clients_by_membership(membership, ..."
1,"INSERT INTO clients_by_membership(membership, ..."
2,"INSERT INTO clients_by_membership(membership, ..."
3,"INSERT INTO clients_by_membership(membership, ..."
4,"INSERT INTO clients_by_membership(membership, ..."


In [93]:
final_df.to_csv('s-05-insert-clients-by-membership.cql', index=False)

# queries_keyspace

## s-06-insert-favorite-category-by-client.cql

In [8]:
clients_df["CLIENTE"] = clients_df["CLIENTE"].str.lower()
clients_df.head()

,CLIENTE,NOMBRE,APELLIDO,PAIS,MEMBRESIA,TELEFONO
0,alfonso,Alfonso,Barajas,Mexico,Estándar,5539848129
1,antonio,Antonio,Aguilar,México,Estándar,5539299465
2,artemio,Artemio,Padilla,Nigeria,Diamante,5539852382
3,avilix,Avilix,Hernández,México,Estándar,5539852756
4,benito,Benito,Franco,Nigeria,Estándar,5539448736


In [104]:
df["CLIENTE"] = df["CLIENTE"].str.lower()
df.head()

,CLIENTE,PRODUCTO,AUTOR,CALIFICACION,CATEGORIA
0,alfonso,Un mundo feliz,Aldous Huxley,10,ficcion
1,alfonso,La dama de las camelias,Alexandre Dumas,10,novela
2,alfonso,El lazarillo de Tormes,Anónimo,9,novela
3,alfonso,El Principito,Antoine de Sainte-Exupéry,10,fantasia
4,alfonso,Nacidos para correr,Christopher McDougall,10,deportes


In [159]:
new_rows = []
base = "INSERT INTO favorite_category_by_client(username, category_frequency, category) VALUES ("
sep = "\', \'"
unique = "\'"
dict_sep = "\', {\'"
end_dict_sep = "\'}"
#--------------------------------------------------------
for idx, row in clients_df.iterrows():
    temp_df = df[df["CLIENTE"] == row["CLIENTE"]].groupby("CATEGORIA").count().reset_index().loc[:, ["CATEGORIA", "CLIENTE"]]
    temp_df = temp_df.rename(columns={'CLIENTE':'FRECUENCIA'})
    for cat, freq in zip(temp_df["CATEGORIA"], temp_df["FRECUENCIA"]):
        values = unique + row["CLIENTE"] + unique + ", " + str(freq) + ", " + unique + cat +unique
        new_rows.append(base + values + ");")
#--------------------------------------------------------
final_df = pd.DataFrame(data = {'statement': new_rows})
print(final_df.iloc[:, :].to_numpy()[:10])
final_df.head()

[["INSERT INTO favorite_category_by_client(username, category_frequency, category) VALUES ('alfonso', 2, 'ajedrez');"]
 ["INSERT INTO favorite_category_by_client(username, category_frequency, category) VALUES ('alfonso', 2, 'autoayuda');"]
 ["INSERT INTO favorite_category_by_client(username, category_frequency, category) VALUES ('alfonso', 2, 'deportes');"]
 ["INSERT INTO favorite_category_by_client(username, category_frequency, category) VALUES ('alfonso', 4, 'fantasia');"]
 ["INSERT INTO favorite_category_by_client(username, category_frequency, category) VALUES ('alfonso', 2, 'ficcion');"]
 ["INSERT INTO favorite_category_by_client(username, category_frequency, category) VALUES ('alfonso', 1, 'finanzas');"]
 ["INSERT INTO favorite_category_by_client(username, category_frequency, category) VALUES ('alfonso', 15, 'novela');"]
 ["INSERT INTO favorite_category_by_client(username, category_frequency, category) VALUES ('alfonso', 1, 'poesia');"]
 ["INSERT INTO favorite_category_by_client(u

,statement
0,INSERT INTO favorite_category_by_client(userna...
1,INSERT INTO favorite_category_by_client(userna...
2,INSERT INTO favorite_category_by_client(userna...
3,INSERT INTO favorite_category_by_client(userna...
4,INSERT INTO favorite_category_by_client(userna...


In [160]:
final_df.to_csv('s-06-insert-favorite-category-by-client.cql', index=False)

## s-06-best-books-by-category.cql

In [16]:
categories_df = pd.read_csv('raw_categories.csv')
categories_df.head()

,CATEGORIA,FRECUENCIA
0,ajedrez,2
1,autoayuda,2
2,biografia,2
3,comedia,2
4,deportes,3


In [162]:
df.head()

,CLIENTE,PRODUCTO,AUTOR,CALIFICACION,CATEGORIA
0,alfonso,Un mundo feliz,Aldous Huxley,10,ficcion
1,alfonso,La dama de las camelias,Alexandre Dumas,10,novela
2,alfonso,El lazarillo de Tormes,Anónimo,9,novela
3,alfonso,El Principito,Antoine de Sainte-Exupéry,10,fantasia
4,alfonso,Nacidos para correr,Christopher McDougall,10,deportes


In [165]:
new_rows = []
base = "INSERT INTO best_books_by_category(book_category, score, book_title, authors) VALUES ("
sep = "\', \'"
unique = "\'"
dict_sep = "\', {\'"
end_dict_sep = "\'}"
#--------------------------------------------------------
for idx, row in categories_df.iterrows():
    category = row["CATEGORIA"]
    temp_df = df[df["CATEGORIA"] == category]
    for t_idx, t_row in temp_df.iterrows():
        values = unique + category + unique + "," + str(t_row["CALIFICACION"]) + "," + unique + t_row["PRODUCTO"] + dict_sep + t_row["AUTOR"] + end_dict_sep
        new_rows.append(base + values + ");")
#--------------------------------------------------------
final_df = pd.DataFrame(data = {'statement': new_rows})
print(final_df.iloc[:, :].to_numpy()[:10])
final_df.head()

[["INSERT INTO best_books_by_category(book_category, score, book_title, authors) VALUES ('ajedrez',10,'El ajedrez de Bobby Fischer', {'Elie Agur'});"]
 ["INSERT INTO best_books_by_category(book_category, score, book_title, authors) VALUES ('ajedrez',9,'Ajedrez genial', {'Manuel López Michelone'});"]
 ["INSERT INTO best_books_by_category(book_category, score, book_title, authors) VALUES ('autoayuda',10,'Hábitos atómicos', {'James Clear'});"]
 ["INSERT INTO best_books_by_category(book_category, score, book_title, authors) VALUES ('autoayuda',10,'Los 7 hábitos de la gente altamente efectiva', {'Stephen R. Covey'});"]
 ["INSERT INTO best_books_by_category(book_category, score, book_title, authors) VALUES ('biografia',9,'Benjamin Franklin', {'Walter Isaacson'});"]
 ["INSERT INTO best_books_by_category(book_category, score, book_title, authors) VALUES ('biografia',9,'Suite francesa', {'Irene Nemirovsky'});"]
 ["INSERT INTO best_books_by_category(book_category, score, book_title, authors) VAL

,statement
0,INSERT INTO best_books_by_category(book_catego...
1,INSERT INTO best_books_by_category(book_catego...
2,INSERT INTO best_books_by_category(book_catego...
3,INSERT INTO best_books_by_category(book_catego...
4,INSERT INTO best_books_by_category(book_catego...


In [166]:
final_df.to_csv('s-06-insert-best_books_by_category.cql', index=False)

## s-06-most-satisfied-clients-by-book.cql

In [167]:
df.head()

,CLIENTE,PRODUCTO,AUTOR,CALIFICACION,CATEGORIA
0,alfonso,Un mundo feliz,Aldous Huxley,10,ficcion
1,alfonso,La dama de las camelias,Alexandre Dumas,10,novela
2,alfonso,El lazarillo de Tormes,Anónimo,9,novela
3,alfonso,El Principito,Antoine de Sainte-Exupéry,10,fantasia
4,alfonso,Nacidos para correr,Christopher McDougall,10,deportes


In [12]:
new_rows = []
base = "INSERT INTO most_satisfied_clients_by_book(book_title, score, username, country, membership, authors, book_category) VALUES ("
sep = "\', \'"
unique = "\'"
dict_sep = "\', {\'"
end_dict_sep = "\'}"
#--------------------------------------------------------
for idx, row in df.iterrows():
    temp_df = clients_df[clients_df["CLIENTE"] == row["CLIENTE"].replace(' ', '')].loc[:, ["PAIS", "MEMBRESIA"]]
    country = temp_df["PAIS"].values[0]
    membership = temp_df["MEMBRESIA"].values[0]
    values = unique + row["PRODUCTO"] + unique + "," + str(row["CALIFICACION"]) + "," + unique + row["CLIENTE"] + sep + country + sep + membership + dict_sep + row["AUTOR"] + end_dict_sep + "," + unique + row["CATEGORIA"] + unique
    new_rows.append(base + values + ");")
#--------------------------------------------------------
final_df = pd.DataFrame(data = {'statement': new_rows})
print(final_df.iloc[:, :].to_numpy()[:2])
final_df.head()

[["INSERT INTO most_satisfied_clients_by_book(book_title, score, username, country, membership, authors, category) VALUES ('Un mundo feliz',10,'alfonso', 'Mexico', 'Estándar', {'Aldous Huxley'},'ficcion');"]
 ["INSERT INTO most_satisfied_clients_by_book(book_title, score, username, country, membership, authors, category) VALUES ('La dama de las camelias',10,'alfonso', 'Mexico', 'Estándar', {'Alexandre Dumas'},'novela');"]]


,statement
0,INSERT INTO most_satisfied_clients_by_book(boo...
1,INSERT INTO most_satisfied_clients_by_book(boo...
2,INSERT INTO most_satisfied_clients_by_book(boo...
3,INSERT INTO most_satisfied_clients_by_book(boo...
4,INSERT INTO most_satisfied_clients_by_book(boo...


In [13]:
final_df.to_csv('s-06-insert-most-satisfied-clients-by-book.cql', index=False)